# Text extraction from  forms using Opencv 

In [2]:
#importing the packages
import cv2
import numpy as np 
import os
import pytesseract
import pandas as pd
pytesseract.pytesseract.tesseract_cmd=r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

         #storing the region of intrest(roi) i.e from which region in the form we have to capture information

roi=[[(29 , 243),(458 , 287),"text","Name"],
[(29 , 351),(460 , 391),"text","Email address"],
[(30 , 457),(249 , 494),"text","Phone_number"],
[(38 , 566),(54 , 582),"box","Executive"],
[(141 , 565),(157 , 581),"box","Sales"],
[(38 , 595),(54 , 610),"box","Marketing"],
[(142 , 594),(158 , 609),"box","Technology"],
[(38 , 623),(54 , 639),"box","ALL"]]
'''roi=[[(27 , 230),(441 , 266),"text","Name"],
    [(28 , 325),(441 , 363),"text","Email address"],
[(25 , 419),(233 , 458),"text","Phone_number"],
[(32 , 519),(47 , 532),"text","Executive"],
[(136 , 519),(151 , 533),"text","Sales"],
[(32 , 548),(47 , 562),"text","Marketing"],
[(135 , 547),(151 , 563),"text","Technology"],
[(31 , 577),(48 , 591),"text","ALL"]]'''

#match percentage 
per=25 

#reading the row or form image using opencv
image_ft=cv2.imread("path",1)

#checking the shape of an image
h,w,c=image_ft.shape

# Reshaping the image if the the image size is too large
image_ft=cv2.resize(image_ft,(w,h))

#ORB(oriented BRIEF) used to detect stable keypoints, selects the strongest features
orb=cv2.ORB_create(1000)

#kp1(keypoiints) are the unique elements to your image and des1(discriptors) are the representation of the keypoints that will be understandable by the computer
kp1,des1=orb.detectAndCompute(image_ft,None)


#this will mark keypoints in the image
imgkp1=cv2.drawKeypoints(image_ft,kp1,None)
#cv2.imshow("img_keypoints",imgkp1)
#path of the form images 
path="path"
mypic= os.listdir(path)
print(mypic)
# creating the empty list
mydata=[]

#iterating the pic and the index value
for j,i in enumerate(mypic):
    print(i)
    image=cv2.imread(path+"\\"+i)
    image=cv2.resize(image,(w,h))
    #cv2.imshow(i,image)
    #cv2.waitKey(0)
    kp2,des2=orb.detectAndCompute(image,None)
    #brudeforce to find the matches between the sorce empty form and all the other filled form images
    b_force=cv2.BFMatcher(cv2.NORM_HAMMING)
    #checking the matches between des1 and des2
    match=b_force.match(des2,des1)
    #it will sort all the match values based on distances
    match.sort(key=lambda x:x.distance)
    #select how many good matches you want
    exact=match[:int(len(match)*(per/100))]
    #Draw the good matches 
    image_match=cv2.drawMatches(image,kp2,image_ft,kp1,exact,None,flags=2)
    image_match=cv2.resize(image_match,(w,h))
    #cv2.imshow(i,image_match)
    #cv2.waitKey(0)
    
    #finding all the source points and the destnation points to find the relationship between these two
    source_points=np.float32([kp2[m.queryIdx].pt for m in exact]).reshape(-1,1,2) 
    dest_points=np.float32([kp1[m.trainIdx].pt for m in exact]).reshape(-1,1,2)
    #finding the relationship is called homogrphafy
    M,_ =cv2.findHomography(source_points,dest_points,cv2.RANSAC,5.0)
    #wrap perspective to get bird eye view of the images i.e to make form images better 
    image_scan=cv2.warpPerspective(image,M,(w,h))
    #image_scan=cv2.resize(image_scan,(w,h))
    #cv2.imshow(i,image_scan)
    
    #Creating the empty image(image_scan) to mask
    image_show=image_scan.copy()
    image_mask=np.zeros_like(image_show)   
    print(f'#######the form {j}')
    pixel_threshold=90
 
    #looping the roi(Region Of Intrest) on each filled image
    for x,r in enumerate(roi):
        #masking the region from where you need to get text
        cv2.rectangle(image_mask,(r[0][0],r[0][1]),(r[1][0],r[1][1]),(0,255,0),cv2.FILLED)
        image_show=cv2.addWeighted(image_show,0.99,image_mask,0.5,0)
        #Croping the fields from all the images that you masked in the previous step 
        image_crop=image_scan[r[0][1]:r[1][1],r[0][0]:r[1][0]]
        #cv2.imshow(str(x),image_crop)
        
        #condition to extract text from roi
        if r[2]=="text":
            #print(f'{r[3]} :{pytesseract.image_to_string(image_crop)}')
            #Extraxting the text from the croped image
            mydata.append(pytesseract.image_to_string(image_crop))
        #print("the data is:"+ str(mydata))

        # conditions to extrat check box information
        if r[2]=="box":
            #Converting color image of form to grayscale image 
            image_gray=cv2.cvtColor(image_crop,cv2.COLOR_BGR2GRAY)
            #Setting the threshold for the pixels  
            image_trash=cv2.threshold(image_gray,200,255,cv2.THRESH_BINARY_INV)[1]
            total_pixels=cv2.countNonZero(image_trash)
            #print(total_pixels)
            if total_pixels>pixel_threshold:total_pixels=1;
            else:total_pixels=0
            #print(f'{r[3]} :{total_pixels}')
            mydata.append(total_pixels)
print(mydata)
splits = np.array_split(mydata,len(mydata)/8)

#Storing the images in the form of dataframe  
for array in splits:
    x=list(array)
    print(x)
    data1=pd.read_csv('C:\\Users\\Dinesh ram\\OneDrive\\Desktop\\data3.csv')    
    df = data1.append(pd.Series(x, index=['Name','Email_address','Phone_number','Executive','Sales','Marketing','Technology','ALL']), ignore_index=True)
    df.to_csv('path',index=False, header=True)

    



['form2.PNG', 'form3.PNG', 'original.PNG']
form2.PNG
#######the form 0
form3.PNG
#######the form 1
original.PNG
#######the form 2
['Dinesh Reddy\n\x0c', 'dineshS78@GMAIL.COM\n\x0c', '09456387289\n\x0c', 0, 0, 0, 1, 1, 'bramhanandam\n\x0c', 'brami@GMAIL.COM\n\x0c', '09900889988\n\x0c', 0, 1, 1, 1, 0, '\x0c', '\x0c', '\x0c', 0, 0, 0, 0, 0]
['Dinesh Reddy\n\x0c', 'dineshS78@GMAIL.COM\n\x0c', '09456387289\n\x0c', '0', '0', '0', '1', '1']
['bramhanandam\n\x0c', 'brami@GMAIL.COM\n\x0c', '09900889988\n\x0c', '0', '1', '1', '1', '0']
['\x0c', '\x0c', '\x0c', '0', '0', '0', '0', '0']


"with open('C:\\Users\\Dinesh ram\\OneDrive\\Desktop\\data2.csv','a+') as f:\n    for k in mydata:\n        f.write((str(k)+'\n'))\n        f.write('\n')\n        f.close()"

In [135]:
#creating the dataframe 
import pandas as pd
data = pd.DataFrame(columns = ['Name','Email_address','Phone_number','Executive','Sales','Marketing','Technology','ALL'])
data.to_csv('path',index=False, header=True)
